In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

import seaborn as sns
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

## Import Drug Review Data

In [ ]:
# Import data
df = pd.read_csv('drugsComTrain_raw.csv')

# Quick Glance
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161297 entries, 0 to 161296
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   uniqueID     161297 non-null  int64 
 1   drugName     161297 non-null  object
 2   condition    160398 non-null  object
 3   review       161297 non-null  object
 4   rating       161297 non-null  int64 
 5   date         161297 non-null  object
 6   usefulCount  161297 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 8.6+ MB


In [ ]:
df.head(10)

,uniqueID,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,3-Nov-15,10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37
5,155963,Cialis,Benign Prostatic Hyperplasia,"""2nd day on 5mg started to work with rock hard...",2,28-Nov-15,43
6,165907,Levonorgestrel,Emergency Contraception,"""He pulled out, but he cummed a bit in me. I t...",1,7-Mar-17,5
7,102654,Aripiprazole,Bipolar Disorde,"""Abilify changed my life. There is hope. I was...",10,14-Mar-15,32
8,74811,Keppra,Epilepsy,""" I Ve had nothing but problems with the Kepp...",1,9-Aug-16,11
9,48928,Ethinyl estradiol / levonorgestrel,Birth Control,"""I had been on the pill for many years. When m...",8,8-Dec-16,1


## Simple Regex Data Cleaning

In [ ]:
# Create a copy of the original df
clean_df = df

# Sub '&#039;' with ' since it's a code for contraction 
clean_df['clean_review'] = clean_df['review'].apply(lambda x: re.sub(r'&#039;', "'", x))

# Sub '&amp;' with 'and' since it's a code for and 
clean_df['clean_review'] = clean_df['clean_review'].apply(lambda x: re.sub(r'&amp;', "and", x))

# Sub '&quo;' with '""' since it's a code for quotations
clean_df['clean_review'] = clean_df['clean_review'].apply(lambda x: re.sub(r'&quot;', '"', x))

# Remove quotations before and after a string
clean_df['clean_review'] = clean_df['clean_review'].apply(lambda x: re.sub(r'^"|"$', "", x))

# Remove extra whitespaces inside the review
clean_df['clean_review'] = clean_df['clean_review'].apply(lambda x: re.sub(r"\s+", " ", x))

# Remove any trailing whitespaces before and after string
clean_df['clean_review'] = clean_df['clean_review'].apply(lambda x: x.strip())


clean_df['clean_review'].head(10)

0    It has no side effect, I take it in combinatio...
1    My son is halfway through his fourth week of I...
2    I used to take another oral contraceptive, whi...
3    This is my first time using any form of birth ...
4    Suboxone has completely turned my life around....
5    2nd day on 5mg started to work with rock hard ...
6    He pulled out, but he cummed a bit in me. I to...
7    Abilify changed my life. There is hope. I was ...
8    I Ve had nothing but problems with the Keppera...
9    I had been on the pill for many years. When my...
Name: clean_review, dtype: object

In [ ]:
# Quick Glance
for index,text in enumerate(clean_df['clean_review'][:11]):
  print('Review %d:\n'%(index+1),text)

Review 1:
 It has no side effect, I take it in combination of Bystolic 5 Mg and Fish Oil
Review 2:
 My son is halfway through his fourth week of Intuniv. We became concerned when he began this last week, when he started taking the highest dose he will be on. For two days, he could hardly get out of bed, was very cranky, and slept for nearly 8 hours on a drive home from school vacation (very unusual for him.) I called his doctor on Monday morning and she said to stick it out a few days. See how he did at school, and with getting up in the morning. The last two days have been problem free. He is MUCH more agreeable than ever. He is less emotional (a good thing), less cranky. He is remembering all the things he should. Overall his behavior is better. We have tried many different medications and so far this is the most effective.
Review 3:
 I used to take another oral contraceptive, which had 21 pill cycle, and was very happy- very light periods, max 5 days, no other side effects. But it c

## More Data Cleaning & Preprocessing

Install contractions package. If there's an error when trying to install contractions, run this in your Anaconda Prompt:

    conda config --add channels conda-forge
    conda install pyahocorasick

Contractions Documentation: https://www.geeksforgeeks.org/nlp-expand-contractions-in-text-processing/

In [ ]:
import nltk
import spacy
import string

#!pip install contractions
import contractions

from nltk.corpus import stopwords                   # Stopwords corpus
from nltk.stem import PorterStemmer                 # Stemmer
from nltk.stem import WordNetLemmatizer             # Lemmatization

from nltk.corpus import wordnet                     # Wordnet for synonyms & antonyms
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize             # Word Tokenizer
from nltk import FreqDist

from collections import Counter

from sklearn.feature_extraction.text import CountVectorizer          #For Bag of words
from sklearn.feature_extraction.text import TfidfVectorizer          #For TF-IDF
#from nltk import ngrams                                              # ngrams
#from gensim.models import Word2Vec                                   #For Word2Vec

In [ ]:
# Expanding Contractions
clean_df['clean_review'] = clean_df['clean_review'].apply(lambda x: contractions.fix(x))

# Remove any punctations
clean_df['clean_review'] = clean_df['clean_review'].apply(lambda x: re.sub(r'[^\w\s]', ' ', x))

# Remove digits and words containing digits
clean_df['clean_review'] = clean_df['clean_review'].apply(lambda x: re.sub(r'\w*\d\w*','', x))

# Lower case
clean_df['clean_review'] = clean_df['clean_review'].apply(lambda x: x.lower())

# Remove extra whitespaces inside the review
clean_df['clean_review'] = clean_df['clean_review'].apply(lambda x: re.sub(r"\s+", " ", x))

# Remove any trailing whitespaces before and after string
clean_df['clean_review'] = clean_df['clean_review'].apply(lambda x: x.strip())

In [ ]:
# Quick Glance
for index,text in enumerate(clean_df['clean_review'][:11]):
  print('Review %d:\n'%(index+1),text)

Review 1:
 it has no side effect i take it in combination of bystolic mg and fish oil
Review 2:
 my son is halfway through his fourth week of intuniv we became concerned when he began this last week when he started taking the highest dose he will be on for two days he could hardly get out of bed was very cranky and slept for nearly hours on a drive home from school vacation very unusual for him i called his doctor on monday morning and she said to stick it out a few days see how he did at school and with getting up in the morning the last two days have been problem free he is much more agreeable than ever he is less emotional a good thing less cranky he is remembering all the things he should overall his behavior is better we have tried many different medications and so far this is the most effective
Review 3:
 i used to take another oral contraceptive which had pill cycle and was very happy very light periods max days no other side effects but it contained hormone gestodene which is n

### Bryan's Note
I have done the basic text cleaning & preprocessing the reviews. Below, you'll see code blocks of removing basic stopwords, stemming, and lemmatizing. We probably need to discuss on how we should remove stopwords (e.g. use basic stopwords or customize stopwords) and select either stemming or lemmatizing.

In [ ]:
# Removing stopwords
#nltk.download('stopwords') ## uncomment if stopwords hasn't been installed yet

STOPWORDS = set(stopwords.words('english')) ## might need to customize stopwords for Edwards ##

def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return ' '.join([word for word in str(text).split() if word not in STOPWORDS])

#clean_df['clean_review'] = clean_df['clean_review'].apply(lambda text: remove_stopwords(text))
#clean_df.head(10)

In [ ]:
# Stemming
stemmer = PorterStemmer()

def stem_words(text):
    return " ".join([stemmer.stem(word) for word in text.split()])

#clean_df['clean_review'] = clean_df['clean_review'].apply(lambda text: stem_words(text))
#clean_df.head(10)

In [ ]:
# Lemmatization
#nltk.download('wordnet') ## uncomment if wordnet hasn't been installed yet
lemmatizer = WordNetLemmatizer()

def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

#clean_df['clean_review'] = clean_df['clean_review'].apply(lambda text: lemmatize_words(text))
#clean_df.head(10)

### Subset the dataset by Top 5 Conditions

1) Depression

2) Birth Control

3) Anxiety

4) Pain

5) Acne

In [ ]:
# Subset the drug dataset by the Top 5 Conditions
top5 = ['Depression', 'Birth Control', 'Anxiety', 'Pain', 'Acne']
top5_conditions = clean_df[clean_df['condition'].isin(top5)].reset_index()

# Double-Check
#top5_conditions['condition'].unique()
#top5_conditions.info()
#top5_conditions.shape ## 55,494 rows & 9 columns

display(top5_conditions.head(5))

,index,uniqueID,drugName,condition,review,rating,date,usefulCount,clean_review
0,2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17,i used to take another oral contraceptive whic...
1,3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,3-Nov-15,10,this is my first time using any form of birth ...
2,9,48928,Ethinyl estradiol / levonorgestrel,Birth Control,"""I had been on the pill for many years. When m...",8,8-Dec-16,1,i had been on the pill for many years when my ...
3,11,75612,L-methylfolate,Depression,"""I have taken anti-depressants for years, with...",10,9-Mar-17,54,i have taken anti depressants for years with s...
4,14,98494,Nexplanon,Birth Control,"""Started Nexplanon 2 months ago because I have...",3,7-Aug-14,10,started nexplanon months ago because i have a ...


In [ ]:
pd.set_option("display.max_rows", None, "display.max_columns", None)


# We should filter the number of reviews per drug
group_drugs = top5_conditions.groupby(["drugName", 'condition']).agg({"uniqueID":"count", "rating":"mean"}).reset_index()


group_drugs = group_drugs.rename(columns = {'uniqueID':'DrugCount', 'rating':'AvgRating'})
group_drugs.sort_values('DrugCount',ascending = False)

,drugName,condition,DrugCount,AvgRating
219,Etonogestrel,Birth Control,3314,5.808992
214,Ethinyl estradiol / norethindrone,Birth Control,2337,5.658537
406,Nexplanon,Birth Control,2149,5.679851
301,Levonorgestrel,Birth Control,2129,7.061062
211,Ethinyl estradiol / levonorgestrel,Birth Control,1600,5.903125
216,Ethinyl estradiol / norgestimate,Birth Control,1580,5.933544
263,Implanon,Birth Control,1095,6.073059
379,Mirena,Birth Control,965,6.957513
535,Skyla,Birth Control,822,7.255474
318,Lo Loestrin Fe,Birth Control,667,6.053973


In [ ]:
filter_20_more_reviews = group_drugs[group_drugs['DrugCount'] >= 20]
drug_name = filter_20_more_reviews['drugName'].tolist()

In [ ]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

# Number of unique drugs by condition
top5_conditions.groupby(['drugName', 'condition'])['uniqueID'].count().sort_values(ascending = False)

drugName                                                 condition    
Etonogestrel                                             Birth Control    3314
Ethinyl estradiol / norethindrone                        Birth Control    2337
Nexplanon                                                Birth Control    2149
Levonorgestrel                                           Birth Control    2129
Ethinyl estradiol / levonorgestrel                       Birth Control    1600
Ethinyl estradiol / norgestimate                         Birth Control    1580
Implanon                                                 Birth Control    1095
Mirena                                                   Birth Control     965
Skyla                                                    Birth Control     822
Lo Loestrin Fe                                           Birth Control     667
Ethinyl estradiol / etonogestrel                         Birth Control     631
NuvaRing                                                 Bir

In [ ]:
# Filter the drugNames
top5_conditions_filter = top5_conditions[top5_conditions['drugName'].isin(drug_name)].reset_index()

#top5_conditions_filter.shape ## (53741, 9)
#top5_conditions_filter['drugName'].nunique() ## 246 unique drugs

top5_conditions_filter.head()

,level_0,index,uniqueID,drugName,condition,review,rating,date,usefulCount,clean_review
0,1,3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,3-Nov-15,10,this is my first time using any form of birth ...
1,2,9,48928,Ethinyl estradiol / levonorgestrel,Birth Control,"""I had been on the pill for many years. When m...",8,8-Dec-16,1,i had been on the pill for many years when my ...
2,3,11,75612,L-methylfolate,Depression,"""I have taken anti-depressants for years, with...",10,9-Mar-17,54,i have taken anti depressants for years with s...
3,4,14,98494,Nexplanon,Birth Control,"""Started Nexplanon 2 months ago because I have...",3,7-Aug-14,10,started nexplanon months ago because i have a ...
4,5,22,227020,Etonogestrel,Birth Control,"""Nexplanon does its job. I can have worry free...",9,11-Aug-14,11,nexplanon does its job i can have worry free s...


## Baseline Sentiment Score

### Textblob

Run polarity and subjectivity via Textblob

In [ ]:
# Create function to calculate polarity and subjectivity
def detect_tb_polarity(text):
    return TextBlob(text).sentiment.polarity

def detect_tb_subjectivity(text):
    return TextBlob(text).sentiment.subjectivity

In [ ]:
top5_conditions_filter['tb_polarity'] = top5_conditions_filter.clean_review.apply(detect_tb_polarity)
top5_conditions_filter['tb_subjectivity'] = top5_conditions_filter.clean_review.apply(detect_tb_subjectivity)

top5_conditions_filter.head(10)

,level_0,index,uniqueID,drugName,condition,review,rating,date,usefulCount,clean_review,tb_polarity,tb_subjectivity
0,1,3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,3-Nov-15,10,this is my first time using any form of birth ...,0.179545,0.665909
1,2,9,48928,Ethinyl estradiol / levonorgestrel,Birth Control,"""I had been on the pill for many years. When m...",8,8-Dec-16,1,i had been on the pill for many years when my ...,0.079167,0.373958
2,3,11,75612,L-methylfolate,Depression,"""I have taken anti-depressants for years, with...",10,9-Mar-17,54,i have taken anti depressants for years with s...,0.275000,0.525758
3,4,14,98494,Nexplanon,Birth Control,"""Started Nexplanon 2 months ago because I have...",3,7-Aug-14,10,started nexplanon months ago because i have a ...,0.021539,0.499872
4,5,22,227020,Etonogestrel,Birth Control,"""Nexplanon does its job. I can have worry free...",9,11-Aug-14,11,nexplanon does its job i can have worry free s...,0.193750,0.773611
5,6,31,96233,Sertraline,Depression,"""1 week on Zoloft for anxiety and mood swings....",8,7-May-11,3,week on zoloft for anxiety and mood swings i t...,-0.055556,0.522222
6,7,32,204999,Toradol,Pain,"""I am 30 years old. I had a multiple composite...",10,11-Feb-13,16,i am years old i had a multiple composite spin...,-0.228125,0.388542
7,8,37,93678,Morphine,Pain,"""I have been on morphine for at least 7 years....",8,8-May-10,19,i have been on morphine for at least years it ...,-0.150000,0.700000
8,9,44,121333,Venlafaxine,Depression,"""my gp started me on Venlafaxine yesterday to ...",4,27-Apr-16,3,my gp started me on venlafaxine yesterday to h...,-0.087546,0.708120
9,10,52,69629,Buprenorphine,Pain,"""My pain management doctor put me on Butrans p...",8,24-Jun-11,125,my pain management doctor put me on butrans pa...,0.294444,0.454722


### Vader

Run based sentiment analysis to calculate positive and negative scores

In [ ]:
# Create function to calculate positive and negative
def detect_vader_pos(text):
    return vader.polarity_scores(text)['pos']

def detect_vader_neg(text):
    return vader.polarity_scores(text)['neg']

In [ ]:
vader = SentimentIntensityAnalyzer()
top5_conditions_filter['vader_pos'] = top5_conditions_filter.clean_review.apply(detect_vader_pos)
top5_conditions_filter['vader_neg'] = top5_conditions_filter.clean_review.apply(detect_vader_neg)

top5_conditions_filter.head(10)

,level_0,index,uniqueID,drugName,condition,review,rating,date,usefulCount,clean_review,vader_pos,vader_neg
0,1,3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,3-Nov-15,10,this is my first time using any form of birth ...,0.090,0.027
1,2,9,48928,Ethinyl estradiol / levonorgestrel,Birth Control,"""I had been on the pill for many years. When m...",8,8-Dec-16,1,i had been on the pill for many years when my ...,0.163,0.166
2,3,11,75612,L-methylfolate,Depression,"""I have taken anti-depressants for years, with...",10,9-Mar-17,54,i have taken anti depressants for years with s...,0.200,0.231
3,4,14,98494,Nexplanon,Birth Control,"""Started Nexplanon 2 months ago because I have...",3,7-Aug-14,10,started nexplanon months ago because i have a ...,0.082,0.109
4,5,22,227020,Etonogestrel,Birth Control,"""Nexplanon does its job. I can have worry free...",9,11-Aug-14,11,nexplanon does its job i can have worry free s...,0.080,0.063
5,6,31,96233,Sertraline,Depression,"""1 week on Zoloft for anxiety and mood swings....",8,7-May-11,3,week on zoloft for anxiety and mood swings i t...,0.068,0.141
6,7,32,204999,Toradol,Pain,"""I am 30 years old. I had a multiple composite...",10,11-Feb-13,16,i am years old i had a multiple composite spin...,0.000,0.154
7,8,37,93678,Morphine,Pain,"""I have been on morphine for at least 7 years....",8,8-May-10,19,i have been on morphine for at least years it ...,0.000,0.084
8,9,44,121333,Venlafaxine,Depression,"""my gp started me on Venlafaxine yesterday to ...",4,27-Apr-16,3,my gp started me on venlafaxine yesterday to h...,0.131,0.143
9,10,52,69629,Buprenorphine,Pain,"""My pain management doctor put me on Butrans p...",8,24-Jun-11,125,my pain management doctor put me on butrans pa...,0.137,0.111


Calculate prediction from vader then calculate precision

In [ ]:
#prediction
top5_conditions_filter['vader_result'] = np.where(top5_conditions_filter["vader_pos"] > top5_conditions_filter["vader_neg"], 1, 0)

#actual
top5_conditions_filter["rating_positive"] = np.where(top5_conditions_filter['rating'] >= 7, 1, 0)

#top5_conditions_filter.head()

#results
top5_conditions_filter['pred_result'] = np.where((top5_conditions_filter['vader_result'] == 1) &  
                                                 (top5_conditions_filter['rating_positive'] == 1), 'TP', 
                                                np.where((top5_conditions_filter['vader_result'] == 0) &  
                                                 (top5_conditions_filter['rating_positive'] == 1), 'FN', 
                                                        np.where((top5_conditions_filter['vader_result'] == 0) &  
                                                 (top5_conditions_filter['rating_positive'] == 0), 'TN', 'FP')))

top5_conditions_filter.head(30)


,level_0,index,uniqueID,drugName,condition,review,rating,date,usefulCount,clean_review,vader_pos,vader_neg,vader_result,rating_positive,pred_result
0,1,3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,3-Nov-15,10,this is my first time using any form of birth ...,0.090,0.027,1,1,TP
1,2,9,48928,Ethinyl estradiol / levonorgestrel,Birth Control,"""I had been on the pill for many years. When m...",8,8-Dec-16,1,i had been on the pill for many years when my ...,0.163,0.166,0,1,FN
2,3,11,75612,L-methylfolate,Depression,"""I have taken anti-depressants for years, with...",10,9-Mar-17,54,i have taken anti depressants for years with s...,0.200,0.231,0,1,FN
3,4,14,98494,Nexplanon,Birth Control,"""Started Nexplanon 2 months ago because I have...",3,7-Aug-14,10,started nexplanon months ago because i have a ...,0.082,0.109,0,0,TN
4,5,22,227020,Etonogestrel,Birth Control,"""Nexplanon does its job. I can have worry free...",9,11-Aug-14,11,nexplanon does its job i can have worry free s...,0.080,0.063,1,1,TP
5,6,31,96233,Sertraline,Depression,"""1 week on Zoloft for anxiety and mood swings....",8,7-May-11,3,week on zoloft for anxiety and mood swings i t...,0.068,0.141,0,1,FN
6,7,32,204999,Toradol,Pain,"""I am 30 years old. I had a multiple composite...",10,11-Feb-13,16,i am years old i had a multiple composite spin...,0.000,0.154,0,1,FN
7,8,37,93678,Morphine,Pain,"""I have been on morphine for at least 7 years....",8,8-May-10,19,i have been on morphine for at least years it ...,0.000,0.084,0,1,FN
8,9,44,121333,Venlafaxine,Depression,"""my gp started me on Venlafaxine yesterday to ...",4,27-Apr-16,3,my gp started me on venlafaxine yesterday to h...,0.131,0.143,0,0,TN
9,10,52,69629,Buprenorphine,Pain,"""My pain management doctor put me on Butrans p...",8,24-Jun-11,125,my pain management doctor put me on butrans pa...,0.137,0.111,1,1,TP


In [ ]:
temp_list= list(top5_conditions_filter.pred_result)
output = {}
for ele in temp_list:
    if len(output[ele]) >= 0:
        output[ele] =1
    else:
        output[ele] += 1

KeyError: 'TP'

In [ ]:
temp_list= list(top5_conditions_filter.pred_result)
output = {}
for ele in temp_list:
    if output.has_key(ele):
        output[ele] += 1
    else:
        output[ele] = 1 

AttributeError: 'dict' object has no attribute 'has_key'

In [ ]:
#calculate precision TP/ FP + TP
top5_conditions_filter[top5_conditions_filter['pred_result'] == 'TP',]

TypeError: '(0         True
1        False
2        False
3        False
4         True
5        False
6        False
7        False
8        False
9         True
10       False
11       False
12       False
13       False
14       False
15       False
16        True
17        True
18       False
19        True
20        True
21       False
22       False
23       False
24       False
25       False
26       False
27       False
28       False
29        True
30        True
31       False
32       False
33       False
34        True
35       False
36       False
37        True
38        True
39       False
40       False
41        True
42        True
43       False
44       False
45       False
46       False
47       False
48       False
49        True
50       False
51       False
52       False
53       False
54       False
55       False
56       False
57       False
58       False
59       False
60        True
61       False
62        True
63       False
64        True
65       Fal

In [ ]:
top5_conditions_filter["rating_positive"][1]

1

# Spacy

In [ ]:
from spacy import displacy

nlp = spacy.load("en_core_web_sm")
example = top5_conditions_filter['clean_review'][0]
doc = nlp(example)

#displacy.render(doc, style="ent")

In [ ]:
#for token in doc:
#    print(token.text, token.head.text,
#            [child for child in token.children])
nlp = spacy.load('en_core_web_sm')
#doc = nlp('Credit and mortgage account holders must submit their requests')
doc = nlp("I used to take another oral contraceptive, which had 21 pill cycle, and was very happy- very light periods, max 5 days, no other side effects. But it contained hormone gestodene, which is not available in US, so I switched to Lybrel, because the ingredients are similar. When my other pills ended, I started Lybrel immediately, on my first day of period, as the instructions said. And the period lasted for two weeks. When taking the second pack- same two weeks. And now, with third pack things got even worse- my third period lasted for two weeks and now it's the end of the third week- I still have daily brown discharge. The positive side is that I didn't have any other side effects. The idea of being period free was so tempting... Alas")
#doc = nlp('I hate this drug.')

root = [token for token in doc if token.head == token][0]
display(root)

print("Subject:")
for i in range(0,len(list(root.lefts))):
    #subject = list(root.lefts)[i]
    print(list(root.lefts)[i])
print("Predicate:")
for j in range(0,len(list(root.rights))):
    #predicate = list(root.rights)[i]
    print(list(root.rights)[i])
#for descendant in subject.subtree:
#    assert subject is descendant or subject.is_ancestor(descendant)
#    print(descendant.text, descendant.dep_, descendant.n_lefts,
#            descendant.n_rights,
#            [ancestor.text for ancestor in descendant.ancestors])

display(root)


used

Subject:
I
Predicate:
take
take
take
take
take
take


used

In [ ]:
top5_conditions_filter['review'][0]

'"This is my first time using any form of birth control. I&#039;m glad I went with the patch, I have been on it for 8 months. At first It decreased my libido but that subsided. The only downside is that it made my periods longer (5-6 days to be exact) I used to only have periods for 3-4 days max also made my cramps intense for the first two days of my period, I never had cramps before using birth control. Other than that in happy with the patch"'

In [ ]:
# Quick Glance
for index,text in enumerate(top5_conditions_filter['clean_review'][:11]):
  print('Review %d:\n'%(index+1),text)

Review 1:
 this is my first time using any form of birth control i am glad i went with the patch i have been on it for months at first it decreased my libido but that subsided the only downside is that it made my periods longer days to be exact i used to only have periods for days max also made my cramps intense for the first two days of my period i never had cramps before using birth control other than that in happy with the patch
Review 2:
 i had been on the pill for many years when my doctor changed my rx to chateal it was as effective it really did help me by completely clearing my acne this takes about months though i did not gain extra weight or develop any emotional health issues i stopped taking it bc i started using a more natural method of birth control but started to take it bc i hate that my acne came back at age i really hope symptoms like depression or weight gain do not begin to affect me as i am older now i am also naturally moody so this may worsen things i was in a ne

### Tagging NLKT and Replacing Certian Words with Synonyms

For drug reviews especially side effects, Prof Vibs suggest to use a Python library that deals with synonyms. Wordnet from NLKT is one of the options. Should we discuss this method?

Link to NLTK POS Tagging Acronym:
https://www.guru99.com/pos-tagging-chunking-nltk.html#:~:text=POS%20Tagging%20in%20NLTK%20is,each%20word%20of%20the%20sentence.

Example of using NLTK to replace all words in a string with their synonyms:
https://gist.github.com/Ghost---Shadow/c361f2d6b4501f40648b

In [ ]:
# Using the 1st review of top5_conditions as an example

words = word_tokenize(top5_conditions.clean_review[0])
tagged = nltk.pos_tag(words)


for x in tagged:
    print(x)

('i', 'NN')
('used', 'VBN')
('to', 'TO')
('take', 'VB')
('another', 'DT')
('oral', 'JJ')
('contraceptive', 'NN')
('which', 'WDT')
('had', 'VBD')
('pill', 'NN')
('cycle', 'NN')
('and', 'CC')
('was', 'VBD')
('very', 'RB')
('happy', 'JJ')
('very', 'RB')
('light', 'JJ')
('periods', 'NNS')
('max', 'VBP')
('days', 'NNS')
('no', 'DT')
('other', 'JJ')
('side', 'NN')
('effects', 'NNS')
('but', 'CC')
('it', 'PRP')
('contained', 'VBD')
('hormone', 'NN')
('gestodene', 'NN')
('which', 'WDT')
('is', 'VBZ')
('not', 'RB')
('available', 'JJ')
('in', 'IN')
('us', 'PRP')
('so', 'RB')
('i', 'RB')
('switched', 'VBD')
('to', 'TO')
('lybrel', 'VB')
('because', 'IN')
('the', 'DT')
('ingredients', 'NNS')
('are', 'VBP')
('similar', 'JJ')
('when', 'WRB')
('my', 'PRP$')
('other', 'JJ')
('pills', 'NNS')
('ended', 'VBD')
('i', 'RB')
('started', 'VBN')
('lybrel', 'NN')
('immediately', 'RB')
('on', 'IN')
('my', 'PRP$')
('first', 'JJ')
('day', 'NN')
('of', 'IN')
('period', 'NN')
('as', 'IN')
('the', 'DT')
('instructio

In [ ]:
display(top5_conditions.clean_review[0])

'i used to take another oral contraceptive which had pill cycle and was very happy very light periods max days no other side effects but it contained hormone gestodene which is not available in us so i switched to lybrel because the ingredients are similar when my other pills ended i started lybrel immediately on my first day of period as the instructions said and the period lasted for two weeks when taking the second pack same two weeks and now with third pack things got even worse my third period lasted for two weeks and now it is the end of the third week i still have daily brown discharge the positive side is that i did not have any other side effects the idea of being period free was so tempting alas'

In [ ]:
# Find a set of synonyms based on the 1st review of the top5_condition

#words = []
STOPWORDS = set(stopwords.words('english')) # REMOVE STOPWORDS
synonyms = [] # Create a list of synonyms

for word in top5_conditions.clean_review[0].split():
    if word not in STOPWORDS:
        for syn in wordnet.synsets(word):
            for l in syn.lemmas():
                synonyms.append(l.name())

print(set(synonyms))
#display(len(set(synonyms))) ## 741 synonym words

{'enounce', 'education', 'vex', 'destruction', 'exploited', 'live_on', 'brightness_level', 'claim', 'hormone', 'tack', 'day', 'ii', 'calorie-free', 'max', 'hold_in', 'standardized', 'tough', 'placid', '2', 'mob', 'positive', 'element', 'perch', 'hush', 'everyday', 'ignitor', 'incur', 'come', "get_under_one's_skin", 'menses', 'load_down', 'approximation', 'positivist', 'lightness', 'set_off', 'Christ_Within', 'sound_out', 'submit', 'buzz_off', 'silence', 'invite', 'endorsement', 'days', 'start_up', 'live', 'remnant', 'allege', 'contraceptive', 'assoil', 'multitude', 'flip', 'regular', 'nevertheless', 'point', 'mother', 'gravel', 'closing', 'finish', 'defective', 'regrettably', 'tranquil', 'lease', 'bear', 'exempt', 'take_off', 'deal', 'first-class_honours_degree', 'lighting', 'throw', 'drive', 'grow', 'unfit', 'lightsome', 'tranquillize', 'wheel', 'convey', 'eve', 'remove', 'unblock', 'tempting', 'lull', 'waiver', 'things', 'even_out', 'number_one', 'train', 'coterie', 'liberate', 'get_